In [1]:
import sys
import json
import csv
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from collections import Counter
from sklearn.base import TransformerMixin
from scipy.sparse import *

In [2]:
businessjson = 'dataset/business.json'
checkinjson = 'dataset/checkin.json'

# Load Business data
business_json_lines = [json.loads( l.strip() ) for l in open(businessjson).readlines() ]
df = json_normalize(business_json_lines)



In [ ]:
print df['categories'].head(1)

In [3]:
# Get records from Yelp set for only restaurants
df = df[df['categories'].astype(str).str.lower().str.contains('restaurant') == True]
df = df.reset_index(drop=True)


#df = df[df['review_count']>=35]
#df = df.reset_index(drop=True)

#Remove Columns of least significance
df = df.drop(['address','attributes.AcceptsInsurance','attributes.AgesAllowed','attributes.BYOB','attributes.ByAppointmentOnly','attributes.Corkage','attributes.DietaryRestrictions.dairy-free','attributes.DietaryRestrictions.gluten-free','attributes.DietaryRestrictions.halal','attributes.DietaryRestrictions.kosher','attributes.DietaryRestrictions.soy-free','attributes.DietaryRestrictions.vegan','attributes.DietaryRestrictions.vegetarian','attributes.HairSpecializesIn.africanamerican','attributes.HairSpecializesIn.asian','attributes.HairSpecializesIn.coloring','attributes.HairSpecializesIn.curly','attributes.HairSpecializesIn.extensions','attributes.HairSpecializesIn.kids','attributes.HairSpecializesIn.perms','attributes.HairSpecializesIn.straightperms','attributes.Open24Hours','attributes.RestaurantsCounterService','business_id','categories','hours.Friday','hours.Monday','hours.Saturday','hours.Sunday','hours.Thursday','hours.Tuesday','hours.Wednesday','latitude','longitude','name','neighborhood','state','city','postal_code'],axis=1)
df = df.copy()
# Classify the records
df['target'] = [1 if df['stars'][x] >= 3.5 and df['review_count'][x] >= 30 else 0 for x in range(len(df))]
df.to_csv('business_csv_filter.csv', encoding='utf-8', index=False)
#print df_binary['target'].value_counts()



In [ ]:
headers = list(df.select_dtypes(include=['object']).columns.values)
#print headers
df = pd.get_dummies(data=df, columns=headers)
df = df.fillna(0)


#print df.head(10)

In [4]:
from sklearn.preprocessing import LabelEncoder
headers = list(df.select_dtypes(include=['object']).columns.values)
#df = df.fillna('Dummy')
#print headers

#for col in headers:
#print df['attributes.Alcohol'].head(2)
lb_make = LabelEncoder()
for col in headers:
    print col
    df[col] = df[col].fillna('Dummy')

    df[col] = lb_make.fit_transform(df[col])
    print df[col].head(5)
#df['attributes.Ambience.casual'] = lb_make.fit_transform(df['attributes.Ambience.casual'])


print df['attributes.Ambience.casual']

attributes.Alcohol
0    0
1    3
2    2
3    3
4    3
Name: attributes.Alcohol, dtype: int64
attributes.Ambience.casual
0    0
1    0
2    0
3    2
4    0
Name: attributes.Ambience.casual, dtype: int64
attributes.Ambience.classy
0    0
1    0
2    0
3    2
4    0
Name: attributes.Ambience.classy, dtype: int64
attributes.Ambience.divey
0    0
1    0
2    0
3    2
4    1
Name: attributes.Ambience.divey, dtype: int64
attributes.Ambience.hipster
0    0
1    0
2    0
3    2
4    0
Name: attributes.Ambience.hipster, dtype: int64
attributes.Ambience.intimate
0    0
1    0
2    0
3    2
4    0
Name: attributes.Ambience.intimate, dtype: int64
attributes.Ambience.romantic
0    0
1    0
2    0
3    2
4    0
Name: attributes.Ambience.romantic, dtype: int64
attributes.Ambience.touristy
0    0
1    0
2    0
3    2
4    0
Name: attributes.Ambience.touristy, dtype: int64
attributes.Ambience.trendy
0    0
1    0
2    0
3    2
4    0
Name: attributes.Ambience.trendy, dtype: int64
attributes.Ambience.ups

In [ ]:
print len(df)

In [5]:
df_multi_train = df.loc[0:41300]
df_multi_train.to_csv('business_le_train.csv', encoding='utf-8', index=False)

df_multi_test = df.loc[41300:]
df_multi_test.to_csv('business_le_test.csv', encoding='utf-8', index=False)


In [ ]:
# Seggregting data to positive and negative buckets.
# Splitting train and test data for binary.
pos_bucket = df.loc[df['target'] == 1].reset_index(drop=True)
neg_bucket = df.loc[df['target'] == 0].reset_index(drop=True)

print len(pos_bucket)
print len(neg_bucket)

df_pos_train = pos_bucket.loc[0:11588]
df_pos_test = pos_bucket.loc[11588:]
df_neg_train = neg_bucket.loc[0:11884]
df_neg_test = neg_bucket.loc[11884:]

#print df_pos_train.shape[0],"df_pos_train"
#print df_pos_test.shape[0],"df_pos_test"
#print df_neg_train.shape[0],"df_neg_train"
#print df_neg_test.shape[0],"df_neg_test"


df_train = pd.concat([df_pos_train, df_neg_train], axis=0)
df_train = df_train.reset_index(drop=True)

#print df_train.shape[0],"df_train"

df_test = pd.concat([df_pos_test, df_neg_test], axis=0)
df_test = df_test.reset_index(drop=True)

df_train.to_csv('business_multiclass_gd_final_train.csv', encoding='utf-8', index=False)
df_test.to_csv('business_multiclass_gd_final_test.csv', encoding='utf-8', index=False)

print len(df_test)
#print df_test.shape[0],"df_test"
#print df_train.loc[0:1]," df_train.loc_ahead"
#print df_train.loc[12589]," df_train.loc_behind"

In [ ]:
dataset = list()
labels = list()
dataset_test = list()
labels_test = list()

#with open('business_binary_le_finally_train.csv', 'r') as f:
with open('business_binary_gd_finally_train.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)

print your_list[0]
for l in your_list:
    dataset.append(l[0:-1])
    labels.append(l[-1])

with open('business_binary_gd_finally_test.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)

print your_list[0]
for l in your_list:
    dataset_test.append(l[0:-1])
    labels_test.append(l[-1])

In [ ]:
print len(dataset)
print len(dataset_test)

In [ ]:
docs = list()

for list in dataset:
    docs.append([int(n) if n != '' else 0 for n in list])
dataset_csr_train = csr_matrix(docs)



In [ ]:
docs_test = []

for list1 in dataset_test:
    docs_test.append([int(n) if n != '' else 0 for n in list1])
dataset_csr_test = csr_matrix(docs_test)

In [ ]:
print dataset_csr_train.shape
print dataset_csr_test.shape

In [ ]:
from sklearn import preprocessing
mat_train = preprocessing.normalize(dataset_csr_train, norm='l2')
mat_test = preprocessing.normalize(dataset_csr_test, norm='l2')


In [ ]:

print mat_train.shape
print labels

In [ ]:
labels_final = list()

for x in labels:
    labels_final.append(str(int(float(x))))
print labels_final

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

ch2_model = SelectKBest(chi2, k=50).fit(mat_train, labels)
X_chi2 = ch2_model.transform(mat_train)
X_chi2_test = ch2_model.transform(mat_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=15, random_state=10)
y_predict_test = clf.fit(X_chi2, labels).predict(X_chi2_test)


In [ ]:

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=15, random_state=10)
y_predict = clf.fit(X_chi2, labels).predict(X_chi2)

In [ ]:
print y_predict[0:100]

In [ ]:
from sklearn.metrics import accuracy_score
print accuracy_score(labels, y_predict)

In [ ]:
from sklearn.metrics import f1_score
print f1_score(labels_test, y_predict_test, average='micro')  